In [1]:
!uv pip install -q spacy

In [2]:
!uv pip install -q nltk gensim pyLDAvis unidecode matplotlib seaborn pandas pyarrow

In [3]:
!uv pip install -q bertopic polars datasets hf_xet

In [4]:
!uv pip install -q https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.7.0/fr_core_news_sm-3.7.0-py3-none-any.whl

In [5]:
#compaatible uniquement avec verison python 3.11 et moins et mettre la mémoire au max
!uv pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64

Audited 2 packages in 15ms
Audited 1 package in 10ms
Audited 1 package in 14ms
Audited 1 package in 5ms


In [6]:
!uv pip install -q langchain-huggingface==0.0.3

In [7]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from cuml.cluster import HDBSCAN
from scipy.cluster import hierarchy as sch
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from cuml.manifold import UMAP
import polars as pl
import spacy
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import tqdm as notebook_tqdm

/home/onyxia/work/TopicModeling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import html 
import json
import matplotlib.pyplot  as plt
import nltk
import numpy as np
import os
import pandas as pd
import unidecode
import re
import requests
import seaborn as sns
import string
import unidecode
import warnings
import torch
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


True

In [9]:
stop_words = set(stopwords.words('french'))

In [10]:
class CachedLemmatizer:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.cache = {}  # Manual cache as a dictionary

    def lemmatize(self, word, pos='n'):
        if word in self.cache:
            return self.cache[word]
        else:
            lemmatized_word = self.lemmatizer.lemmatize(word, pos)
            self.cache[word] = lemmatized_word  # Store in cache
            return lemmatized_word


cached_lemmatizer = CachedLemmatizer()

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
nlp = spacy.load("fr_core_news_sm")  

In [13]:
def preprocess(docs):
    cleaned = []
    for doc in nlp.pipe(docs, batch_size=100, n_process=4):
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        cleaned.append(' '.join(tokens))
    return cleaned

In [14]:
DICTIONNARY =  ['accord','entreprise', 'preambule', 'sommaire',  'code', 'syndical', 'responsable', 'representant', 
                'present', 'ca', 'organisation', 'preambule', 'peut', 'etre', 'contrat','travail', 'ressources','humaines', 'mise',
                'ainsi', 'et', 'ou', 'alors','collaborateur', 'ci', 'apres', 'party', 'signataire', 'tout', 'etat', 'cause', 'societe', 
                'notamment','article','activite', 'cette', 'donc', 'si', 'sous', 'disposition', 'convention', 'collective', 'dans', 'a', 'cadre',
                'signataire', 'partie', 'parties', 'entre', 'doit', 'mme', 'mr', 'madame', 'monsieur'
               ]

DICTIONNARY_STEM = ['part', 'signatair', 'organis', 'syndical', 
                    'dont', 'sieg', 'social', 'conseil', 'prud', 'homm', 
                   'vi', 'professionnel', 'disposit', 'legal', 'conventionnel']

In [15]:
def preprocess_text(text, lang="french"):
    # décoage HTML
    text = html.unescape(text)
    
    # nettoyage de tous les cractères spéciaux
    text = re.sub(r"&[a-z]+;", " ", text)
    text = re.sub(r"&#\d+;", " ", text)
    text = re.sub(r"[<>{}\[\]\|\^\~`\"'=]+", " ", text)
    text = re.sub(r"[–—•«»]+", " ", text)  # Tirets longs, puces, guillemets français

    # tokenisation
    words = word_tokenize(text)

    # lemming
    #stemmer = SnowballStemmer(lang)
                  
    wnl = cached_lemmatizer
   
    words_cleaned = []
    for w in words:
        #w_norm = unidecode.unidecode(w.lower())
        w_norm = w.lower()
        if (
            w_norm not in stop_words
            and w_norm not in DICTIONNARY
            and w_norm not in string.punctuation
            and not re.search(r"[<>]|--+|__+|xx+|==+", w_norm)
            and not w_norm.isnumeric()
            and len(w_norm) > 2
        ):
            words_cleaned.append(wnl.lemmatize(w_norm))
            #words_cleaned.append(stemmer.stem(w_norm))

    return words_cleaned

In [16]:
import re

def normalize(text):
    return text.lower().strip()

def split_text_by_sentences(text, flagged_sentences):
    split_texts = []
    positions = []

    normalized_text = normalize(text)

    # On garde un mapping (titre original, position) pour préserver les titres initiaux
    for sentence in flagged_sentences:
        norm_sentence = normalize(sentence)
        pos = normalized_text.find(norm_sentence)
        if pos != -1:
            # On retrouve la position réelle dans le texte original
            real_pos = text.lower().find(sentence.lower())
            if real_pos != -1:
                positions.append(real_pos)

    # Si aucune position trouvée, retourner le texte complet
    if not positions:
        return [text]

    positions = sorted(set(positions))
    positions.insert(0, 0)
    positions.append(len(text))

    for i in range(len(positions) - 1):
        start = positions[i]
        end = positions[i + 1]
        split_texts.append(text[start:end].strip())

    return split_texts



In [17]:
def split_text_with_titles(text, summary_titles):
    chunks = split_text_by_sentences(text, summary_titles)
    result = {}
    for title in summary_titles:
        for chunk in chunks:
            if normalize(title) in normalize(chunk[:len(title)+30]):
                result[title] = chunk.strip()
                break
    return result


In [18]:
model_kwargs = {'device': 'cuda'} 
#model_kwargs = {'device': 'cpu'}  
MODEL_NAME_EMBEDDER="BAAI/bge-small-en-v1.5"  #petit modèle en anglais
#MODEL_NAME_EMBEDDER="BAAI/bge-m3" #gros modèle multilingue

embedder = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_EMBEDDER, 
    model_kwargs=model_kwargs,
    show_progress=True
)


phrases_non_metier = [
    "Révision de l’accord",
    "Dénonciation de l’accord",
    "Interprétation de l’accord",
    "Suivi de l’accord",
    "Durée de l’accord",
    "Formalités de publicité et de dépôt",
    "Publicité et dépôt",
    "Date d'effet et durée",
    "Champ d'application",
    "Clause de revoyure", 
    "Information des représentants du personnel", 
    "Dispositions relatives à l’accord",
    "Champ d’application",
    "Commission de suivi", 
    "Pause déjeuner du personnel", 
    "Modification de l'accord",
    "Adhésion"
    
]

# Embeddings des phrases non-métier
ref_embeddings = embedder.embed_documents(phrases_non_metier)

def filtre_par_similarite(phrases, seuil=0.85):##torp long utiliser version vectoisée
    results = []
    for phrase in phrases:
        emb = embedder.embed_query(phrase)
        sims = cosine_similarity([emb], ref_embeddings)[0]
        if max(sims) < seuil:
            results.append(phrase) 
    return results

def filtre_par_similarite_vectorise(phrases, seuil=0.85):
    if not phrases:
        return []

    phrase_embeddings = embedder.embed_documents(phrases)  
    sims = cosine_similarity(phrase_embeddings, ref_embeddings)

    # On garde les phrases dont la similarité max avec une phrase non-métier est < seuil
    keep_idx = np.max(sims, axis=1) < seuil
    return [phrase for phrase, keep in zip(phrases, keep_idx) if keep]

    
def filtre_chunks_par_titre(section_dict, phrases_non_metier, seuil=0.85): #seuil arbitraire : en tester plsr
    """
    Ne garde que les chunks dont le titre est peu similaire aux phrases non métier.
    """
    if not section_dict:
        return []

    titres = list(section_dict.keys())
    chunks = list(section_dict.values())

    # Embeddings des titres de section
    titre_embeddings = embedder.embed_documents(titres)
    ref_embeddings = embedder.embed_documents(phrases_non_metier)

    sims = cosine_similarity(titre_embeddings, ref_embeddings)

    # On garde les chunks dont le titre est peu similaire aux phrases non métier
    keep_idx = np.max(sims, axis=1) < seuil
    return [chunk.strip() for chunk, keep in zip(chunks, keep_idx) if keep]



Batches: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


In [19]:
sommaire_hs = pd.read_parquet("data/echantillon_1000_hs_accords_TOC.parquet")
df_hs = pd.read_parquet("data/echantillon_1000_hs_accords.parquet")
df_hs = df_hs.set_index("numdossier_new")
df_hs = df_hs.merge(sommaire_hs,how="inner",left_index=True,right_index=True)
df_hs = df_hs.rename(columns={"extracted_summary":"summary"})

In [20]:
df_hs["section_dict"] = df_hs.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["summary"]),
    axis=1
)

In [21]:
def get_all_chunks(section_dict):
    chunks = list(section_dict.values())
    return [chunk.strip() for chunk in chunks]

In [22]:
def get_valid_chunks_filtered(section_dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85):
    skip_titles_norm = [normalize(t) for t in skip_titles]

    # supprimer le préambule et avant 
    titles = list(section_dict.keys())
    preamble_idx = next((i for i, t in enumerate(titles) if "préambule" in normalize(t)), -1)
    if preamble_idx != -1:
        titles = titles[preamble_idx + 1:]

    # garder les titres valides uniquement
    valid_titles = [
        t for t in titles if all(skip_kw not in normalize(t) for skip_kw in skip_titles_norm)
    ]
    candidate_dict = {t: section_dict[t] for t in valid_titles}

    # filtrer par similarité des titres
    return filtre_chunks_par_titre(candidate_dict, phrases_non_metier, seuil=seuil_sim)


# En filtrant les chunks

In [23]:
df_hs["lda_documents"] = df_hs["section_dict"].apply(get_valid_chunks_filtered)

Batches: 100%|██████████| 1/1 [00:00<00:00, 71.26it/s]


In [ ]:
filtered_chunks_hs = [chunk for doc_chunks in df_hs["lda_documents"] for chunk in doc_chunks]
filtered_docs_cleaned = preprocess(filtered_chunks_hs)

In [ ]:
#Embeddings
start = time.time()
embedding_model = SentenceTransformer("all-MiniLM-L6-v2",device='cuda')  
embeddings_filtered = embedding_model.encode(filtered_docs_cleaned, show_progress_bar=True)
print(f"[1] Embedding en {time.time() - start:.2f}s")

In [ ]:
#Réduction UMAP
start = time.time()
umap_model = UMAP(n_components=5, n_neighbors=50, random_state=42, metric="cosine", verbose=True)
#umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', verbose=True)
umap_embeddings_filtered = umap_model.fit_transform(embeddings_filtered)
print(f"[2] UMAP en {time.time() - start:.2f}s")

In [ ]:
# Clustering 
start = time.time()
hdbscan_model =  HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True, verbose=True)
#hdbscan_model = HDBSCAN(min_samples=20, gen_min_span_tree=True, prediction_data=False, min_cluster_size=20, verbose=True)
clusters = hdbscan_model.fit_predict(umap_embeddings_filtered)
print(f"[3] HDBSCAN en {time.time() - start:.2f}s")
print(f"[3] Nombre de clusters trouvés : {len(np.unique(clusters))}")

In [ ]:
start = time.time()
topic_model_filtered = BERTopic(
    language="french",
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)
topics, probs = topic_model_filtered.fit_transform(filtered_docs_cleaned, embeddings=embeddings_filtered)
print(f"[4] BERTopic final en {time.time() - start:.2f}s")


In [ ]:
#from bertopic import BERTopic
#topic_model_filter = BERTopic(language="french")
#topic_model_filter, probs_no_filter = topic_model_no_filter.fit_transform(all_docs_cleaned)

In [ ]:
topic_model_filtered.visualize_topics()

In [ ]:
topic_model_filtered.visualize_barchart()

# BERTopic (KeyBERTInspired)

In [ ]:
representation_model = KeyBERTInspired()

topic_model_KeyBERTInspired = BERTopic(representation_model=representation_model,
                                       embedding_model=embedding_model,
                                       umap_model=umap_model,
                                       hdbscan_model=hdbscan_model,
                                       verbose=True,
                                       language="french")
topics_KeyBERTInspired, probs_KeyBERTInspired = topic_model_KeyBERTInspired.fit_transform(filtered_docs_cleaned)

In [ ]:
len(filtered_docs_cleaned)

In [ ]:
topic_model_KeyBERTInspired.get_topic_info()

In [ ]:
topic_model_KeyBERTInspired.visualize_topics()

In [ ]:
topic_model_KeyBERTInspired.visualize_barchart()

# BERTopic (MMR)

In [ ]:
representation_model = MaximalMarginalRelevance(diversity=0.3)

topic_model_MMR = BERTopic(representation_model=representation_model,
                                       embedding_model=embedding_model,
                                       umap_model=umap_model,
                                       hdbscan_model=hdbscan_model,
                                       verbose=True,
                                       language="french")
topics_MMR, probs_MMR = topic_model_MMR.fit_transform(filtered_docs_cleaned)

In [ ]:
topic_model_MMR.get_topic_info()

 # Hierarchical topics 

In [ ]:
hierarchical_topics = topic_model_MMR.hierarchical_topics(filtered_docs_cleaned)
hierarchical_topics

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model_MMR.hierarchical_topics(filtered_docs_cleaned, linkage_function=linkage_function)

In [ ]:
topic_model_MMR.visualize_hierarchy(hierarchical_topics=hierarchical_topics)